In [43]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
N = 1000
INIT_EXPOSED = int(N*0.01)

from seirsplus.networks import generate_demographic_contact_network, household_country_data

demographic_graphs, individual_ageGroups, households = generate_demographic_contact_network(
                                                            N=N, demographic_data=household_country_data('US'), 
                                                            distancing_scales=[0.7], isolation_groups=[])

G_baseline   = demographic_graphs['baseline']
G_quarantine = demographic_graphs['distancingScale0.7']

households_indices = [household['indices'] for household in households]

Generated overall age distribution:
0-9: 0.1340	(0.0130 from target)
10-19: 0.1200	(-0.0110 from target)
20-29: 0.1380	(0.0010 from target)
30-39: 0.1260	(-0.0070 from target)
40-49: 0.1270	(0.0030 from target)
50-59: 0.1230	(-0.0080 from target)
60-69: 0.1040	(-0.0110 from target)
70-79: 0.0900	(0.0200 from target)
80+: 0.0380	(0.0000 from target)

Generated household size distribution:
1: 0.2829	(-0.0008 from target)
2: 0.3439	(-0.0012 from target)
3: 0.1537	(0.0030 from target)
4: 0.1220	(-0.0057 from target)
5: 0.0707	(0.0130 from target)
6: 0.0220	(-0.0007 from target)
7: 0.0049	(-0.0076 from target)
Num households: 410
mean household size: 2.448393594

Generating graph for 0-9...
Generating graph for 10-19...


/home/yz685/anaconda3/envs/corr_pooling/lib/python3.11/site-packages/networkx/linalg/graphmatrix.py:187: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return adjacency_matrix(G, nodelist, dtype, weight)


Generating graph for 20-59...
Generating graph for 60+...


In [51]:
from seirsplus.viral_model import ViralExtSEIRNetworkModel

test_model = ViralExtSEIRNetworkModel(
    G = G_baseline,
    beta = 0.1,
    sigma = 0.2,
    lamda = 0.5,
    gamma  = 0.25,
    initE = INIT_EXPOSED)

/home/yz685/anaconda3/envs/corr_pooling/lib/python3.11/site-packages/networkx/linalg/graphmatrix.py:187: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return adjacency_matrix(G, nodelist, dtype, weight)


In [52]:
test_model.tmax=100
test_model.initialize_VL()

In [53]:
# Test:
# run_iteration() should be syntactically correct
# model viral loads should be updated correctly
# get a sense of how much time it takes between transitions

nodes_to_track = set()
last_t = 0

for iter in range(10):
    print('======= Running iteration ', iter, ' ======')
    test_model.run_iteration()
    print('test_model.transitionNode: ', test_model.transitionNode)
    nodes_to_track.add(test_model.transitionNode)
    print('new state: ', test_model.X[test_model.transitionNode])
    print('new viral load: ', test_model.current_VL[test_model.transitionNode])
    print('test_model.t: ', test_model.t, "time since last transition: ", test_model.t - last_t)
    last_t = test_model.t
    print(
        'number of S, E, I_pre, I_sym, I_asym, H, R, F: ',
        [sum(test_model.X == i+1).item() for i in range(8)]
    )
    print(
        'Tracking viral loads of nodes that underwent transitions: ', 
        [(node, test_model.current_VL[node], test_model.current_state_init_VL[node]) for node in nodes_to_track])
    # plt.plot(test_model.current_VL)
    # plt.show()

======= Running iteration  0  ======
test_model.transitionNode:  678
new state:  [3]
new viral load:  6.0
test_model.t:  0.040462975043232444 time since last transition:  0.040462975043232444
number of S, E, I_pre, I_sym, I_asym, H, R, F:  [990, 9, 1, 0, 0, 0, 0, 0]
Tracking viral loads of nodes that underwent transitions:  [(678, 6.0, 6.0)]
======= Running iteration  1  ======
test_model.transitionNode:  730
new state:  [2]
new viral load:  3.0
test_model.t:  0.27822636017003094 time since last transition:  0.2377633851267985
number of S, E, I_pre, I_sym, I_asym, H, R, F:  [989, 10, 1, 0, 0, 0, 0, 0]
Tracking viral loads of nodes that underwent transitions:  [(730, 3.0, 3.0), (678, 6.0, 6.0)]
======= Running iteration  2  ======
test_model.transitionNode:  384
new state:  [3]
new viral load:  6.0
test_model.t:  1.425563929259306 time since last transition:  1.147337569089275
number of S, E, I_pre, I_sym, I_asym, H, R, F:  [989, 9, 2, 0, 0, 0, 0, 0]
Tracking viral loads of nodes that u

## TODO: Run longer, check that infections occur in clusters